Practical 1: Sentiment Detection in Movie Reviews
========================================



This practical concerns detecting sentiment in movie reviews. This is a typical NLP classification task.
In [this file](https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760/reviews.json) (80MB) you will find 1000 positive and 1000 negative **movie reviews**.
Each review is a **document** and consists of one or more sentences.

To prepare yourself for this practical, you should
have a look at a few of these texts to understand the difficulties of
the task: how might one go about classifying the texts? You will write
code that decides whether a movie review conveys positive or
negative sentiment.

Please make sure you have read the following paper:

>   Bo Pang, Lillian Lee, and Shivakumar Vaithyanathan
(2002). 
[Thumbs up? Sentiment Classification using Machine Learning
Techniques](https://dl.acm.org/citation.cfm?id=1118704). EMNLP.

Bo Pang et al. were the "inventors" of the movie review sentiment
classification task, and the above paper was one of the first papers on
the topic. The first version of your sentiment classifier will do
something similar to Pang et al.'s system. If you have questions about it,
you should resolve as soon as possible with your TA.


**Advice**

Please read through the entire practical and familiarise
yourself with all requirements before you start coding or otherwise
solving the tasks. Writing clean and concise code can make the difference
between solving the assignment in a matter of hours, and taking days to
run all experiments.

## Environment

All code should be written in **Python 3**. 
This is the default in Google Colab.

In [1]:
!python --version

Python 3.7.9


If you want to run code on your own computer, then download this notebook through `File -> Download .ipynb`.
The easiest way to
install Python is through downloading
[Anaconda](https://www.anaconda.com/download). 
After installation, you can start the notebook by typing `jupyter notebook filename.ipynb`.
You can also use an IDE
such as [PyCharm](https://www.jetbrains.com/pycharm/download/) to make
coding and debugging easier. It is good practice to create a [virtual
environment](https://docs.python.org/3/tutorial/venv.html) for this
project, so that any Python packages don’t interfere with other
projects. 
 

**Learning Python 3**

If you are new to Python 3, you may want to check out a few of these resources:
- https://learnxinyminutes.com/docs/python3/
- https://www.learnpython.org/
- https://docs.python.org/3/tutorial/

In [2]:
import math
import os
import sys
from subprocess import call
from nltk import FreqDist
from nltk.util import ngrams
from nltk.stem.porter import PorterStemmer
import sklearn as sk
import pickle
import json
from collections import Counter
import requests
import matplotlib.pyplot as plt
import numpy as np

## Loading the data

**Download the sentiment lexicon and the movie reviews dataset.**

In [3]:
import urllib.request
url = 'https://gist.githubusercontent.com/bastings/d6f99dcb6c82231b94b013031356ba05/raw/f80a0281eba8621b122012c89c8b5e2200b39fd6'
filename = 'sent_lexicon'
urllib.request.urlretrieve(url, filename)

('sent_lexicon', <http.client.HTTPMessage at 0x2797e0e4488>)

In [4]:
url = 'https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760'
filename = 'reviews.json'
urllib.request.urlretrieve(url, filename)

('reviews.json', <http.client.HTTPMessage at 0x2797e0e4dc8>)

In [5]:
# download sentiment lexicon
!wget https://gist.githubusercontent.com/bastings/d6f99dcb6c82231b94b013031356ba05/raw/f80a0281eba8621b122012c89c8b5e2200b39fd6/sent_lexicon
# download review data
!wget https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760/reviews.json

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


**Load the movie reviews.**

Each word in a review comes with its part-of-speech tag. For documentation on POS-tags, see https://catalog.ldc.upenn.edu/docs/LDC99T42/tagguid1.pdf.


In [6]:
# file structure:
# [
#  {"cv": integer, "sentiment": str, "content": list} 
#  {"cv": integer, "sentiment": str, "content": list} 
#   ..
# ]
# where `content` is a list of sentences, 
# with a sentence being a list of (token, pos_tag) pairs.


with open("reviews.json", mode="r", encoding="utf-8") as f:
  reviews = json.load(f)
  
print("Total number of reviews:", len(reviews), '\n')

def print_sentence_with_pos(s):
  print(" ".join("%s/%s" % (token, pos_tag) for token, pos_tag in s))

for i, r in enumerate(reviews):
  print(r["cv"], r["sentiment"], len(r["content"]))  # cv, sentiment, num sents
  print_sentence_with_pos(r["content"][0])
  if i == 4: 
    break
    
c = Counter()
for review in reviews:
  for sentence in review["content"]:
    for token, pos_tag in sentence:
      c[token.lower()] += 1
      
print("\nNumber of word types:", len(c))
print("Number of word tokens:", sum(c.values()))

print("\nMost common tokens:")
for token, count in c.most_common(20):
  print("%10s : %8d" % (token, count))
  

Total number of reviews: 2000 

0 NEG 29
Two/CD teen/JJ couples/NNS go/VBP to/TO a/DT church/NN party/NN ,/, drink/NN and/CC then/RB drive/NN ./.
1 NEG 11
Damn/JJ that/IN Y2K/CD bug/NN ./.
2 NEG 24
It/PRP is/VBZ movies/NNS like/IN these/DT that/WDT make/VBP a/DT jaded/JJ movie/NN viewer/NN thankful/JJ for/IN the/DT invention/NN of/IN the/DT Timex/NNP IndiGlo/NNP watch/NN ./.
3 NEG 19
QUEST/NN FOR/IN CAMELOT/NNP ``/`` Quest/NNP for/IN Camelot/NNP ''/'' is/VBZ Warner/NNP Bros./NNP '/POS first/JJ feature-length/JJ ,/, fully-animated/JJ attempt/NN to/TO steal/VB clout/NN from/IN Disney/NNP 's/POS cartoon/NN empire/NN ,/, but/CC the/DT mouse/NN has/VBZ no/DT reason/NN to/TO be/VB worried/VBN ./.
4 NEG 38
Synopsis/NNPS :/: A/DT mentally/RB unstable/JJ man/NN undergoing/VBG psychotherapy/NN saves/VBZ a/DT boy/NN from/IN a/DT potentially/RB fatal/JJ accident/NN and/CC then/RB falls/VBZ in/IN love/NN with/IN the/DT boy/NN 's/POS mother/NN ,/, a/DT fledgling/NN restauranteur/NN ./.

Number of wo

# Lexicon-based approach (2+1pts)



A traditional approach to automatically classify documents according to their sentiment is the lexicon-based approach. To implement this approach, you need a **sentiment lexicon**, i.e., a list of words annotated with a sentiment label (e.g., positive and negative) or a sentiment score (e.g., a score from 0 to 5).

In this practical, you will use the sentiment
lexicon released by Wilson et al. (2005). The path of the loaded lexicon is `"sent_lexicon"`.

> Theresa Wilson, Janyce Wiebe, and Paul Hoffmann
(2005). [Recognizing Contextual Polarity in Phrase-Level Sentiment
Analysis](http://www.aclweb.org/anthology/H/H05/H05-1044.pdf). HLT-EMNLP.

Pay attention to all the information available in the sentiment lexicon. The field *word1* contains the lemma, *priorpolarity* contains the sentiment label (positive, negative, both, or neutral), *type* gives you the magnitude of the word's sentiment (strong or weak), and *pos1* gives you the part-of-speech tag of the lemma. Some lemmas can have multiple part-of-speech tags and thus multiple entries in the lexicon.


Pay attention to all the information available in the sentiment lexicon. The field *word1* contains the lemma, *priorpolarity* contains the sentiment label (positive, negative, both, or neutral), *type* gives you the magnitude of the word's sentiment (strong or weak), and *pos1* gives you the part-of-speech tag of the lemma. Some lemmas can have multiple part-of-speech tags and thus multiple entries in the lexicon.

In [7]:
with open("sent_lexicon", mode="r", encoding="utf-8") as f:
  line_cnt = 0
  for line in f:
    print(line.strip())
    line_cnt += 1
    if line_cnt > 10:
      break

type=weaksubj len=1 word1=abandoned pos1=adj stemmed1=n priorpolarity=negative
type=weaksubj len=1 word1=abandonment pos1=noun stemmed1=n priorpolarity=negative
type=weaksubj len=1 word1=abandon pos1=verb stemmed1=y priorpolarity=negative
type=strongsubj len=1 word1=abase pos1=verb stemmed1=y priorpolarity=negative
type=strongsubj len=1 word1=abasement pos1=anypos stemmed1=y priorpolarity=negative
type=strongsubj len=1 word1=abash pos1=verb stemmed1=y priorpolarity=negative
type=weaksubj len=1 word1=abate pos1=verb stemmed1=y priorpolarity=negative
type=weaksubj len=1 word1=abdicate pos1=verb stemmed1=y priorpolarity=negative
type=strongsubj len=1 word1=aberration pos1=adj stemmed1=n priorpolarity=negative
type=strongsubj len=1 word1=aberration pos1=noun stemmed1=n priorpolarity=negative
type=strongsubj len=1 word1=abhor pos1=anypos stemmed1=y priorpolarity=negative


Given such a sentiment lexicon, there are ways to solve
the classification task without using Machine Learning. For example, one might look up every word $w_1 ... w_n$ in a document, and compute a **binary score**
$S_{binary}$ by counting how many words have a positive or a
negative label in the sentiment lexicon $SLex$.

$$S_{binary}(w_1 w_2 ... w_n) = \sum_{i = 1}^{n}\text{sign}(SLex\big[w_i\big])$$

where $\text{sign}(SLex\big[w_i\big])$ refers to the polarity of $w_i$.

**Threshold.** On average, there are more positive than negative words per review (~7.13 more positive than negative per review) to take this bias into account you should use a threshold of **8** (roughly the bias itself) to make it harder to classify as positive.

$$
\text{classify}(S_{binary}(w_1 w_2 ... w_n)) = \bigg\{\begin{array}{ll}
        \text{positive} & \text{if } S_{binary}(w_1w_2...w_n) > threshold\\
        \text{negative} & \text{otherwise}
        \end{array}
$$


#### (Q1.1) Implement this approach and report its classification accuracy. (1 pt)

In [8]:
### First we create the dataset in a nice format
def create_whole_review(content):
    whole_review = []
    for sentence in content:
        whole_review += [word.lower() for word in np.array(sentence)[:, 0]]
    
    return whole_review

def create_dataset(reviews):
    targets = []
    sentences = []
    for review in reviews:
        targets.append(int( review['sentiment'] == 'POS'))
        
        whole_review = create_whole_review(review['content'])
        sentences.append(whole_review)
    return sentences, targets

sentences, targets = create_dataset(reviews)


In [9]:
### Load the lexixon in a default dict
from collections import defaultdict


sentiments = {
    'positive': 1,
    'negative': -1,
    'both': 0,
    'neutral': 0
}

def get_slex(ref="sent_lexicon"):
    '''
    ref: reference of the file 
    returns: Defaultdictionary{word:  sentiment score} 
    
    '''
    
    slex = defaultdict(float)
    
    ## indexes
    key_index = 2
    key_strip = "word1="
    value_index = 5
    value_strip = "priorpolarity="
    
    magnitude_index = 0
    magnitude_strip = "type="
    
    magnitude_choices = defaultdict(float, {
        "weaksubj": 1.0,
        "strongsubj": 2.0
    })
    
    with open("sent_lexicon", mode="r", encoding="utf-8") as f:
        
        line_cnt = 0
        for line in f:
            line_cnt += 1
            line_arr = line.split()
            magnitude = magnitude_choices[ line_arr[magnitude_index].replace(magnitude_strip, "")]
            sentiment = sentiments[line_arr[value_index].replace(value_strip, "")]
            slex[line_arr[key_index].replace(key_strip, "")] =  magnitude * sentiment
    return slex


    
slex = get_slex()



class BinarySentimentClassifier:
    
    def __init__(self, slex, threshold):
        self.slex = slex
        self.threshold = threshold

    def predict(self, sentences):
        return [int( self.binary_sentiment(sentence) > self.threshold) for sentence in sentences ]
    
    def binary_sentiment(self, sentence):
        '''
        sentence: sentence to calculate the sentiment from
        returns the binary sentiment score of a sentence given the sentiment lexicon
        '''

        return np.sum([ np.sign( self.slex[word])   for word in sentence])



# Create the classifier:
bin_sent_model = BinarySentimentClassifier(slex, 8)

predictions = bin_sent_model.predict(sentences)




In [10]:
print( (np.array(predictions) == np.array(targets)).astype(int)) 

def calculate_results(predictions, targets):
    return np.array(predictions) == np.array(targets)

def calculate_accuracy(predictions, targets):
    return np.sum(calculate_results(predictions, targets)) / len(targets)

[0 1 1 ... 0 1 1]


In [11]:
# token_results should be a list of binary indicators; for example [1, 0, 1, ...] 
# where 1 indicates a correct classification and 0 an incorrect classification.
token_results = calculate_results(predictions, targets)
token_accuracy = calculate_accuracy(predictions, targets)
print("Accuracy: %0.2f" % token_accuracy)

Accuracy: 0.68


As the sentiment lexicon also has information about the **magnitude** of
sentiment (e.g., *“excellent"* would have higher magnitude than
*“good"*), we can take a more fine-grained approach by adding up all
sentiment scores, and deciding the polarity of the movie review using
the sign of the weighted score $S_{weighted}$.

$$S_{weighted}(w_1w_2...w_n) = \sum_{i = 1}^{n}SLex\big[w_i\big]$$


Make sure you define an appropriate threshold for this approach.

#### (Q1.2) Now incorporate magnitude information and report the classification accuracy. Don't forget to use the threshold. (1 pt)

In [12]:
### The calculate the bias we calculate avarage the sentiment of each review. 
avg = 0
total = 0
for sentence in sentences:
    total += np.sum([ slex[word]   for word in sentence])
avg = total / len(sentences)
print(avg)

10.8195


In [13]:

class MagnitudeClassifier:
    
    def __init__(self, slex, threshold):
        self.slex = slex
        self.threshold = threshold

    def predict(self, sentences):
        return [int( self.binary_sentiment(sentence) > self.threshold) for sentence in sentences ]
    
    def binary_sentiment(self, sentence):
        '''
        sentence: sentence to calculate the sentiment from
        returns the binary sentiment score of a sentence given the sentiment lexicon
        '''

        return np.sum([ self.slex[word]   for word in sentence])

magnitude_model = MagnitudeClassifier(slex, avg)

predictions = magnitude_model.predict(sentences)


In [14]:
magnitude_results = calculate_results(predictions, targets)
magnitude_accuracy = calculate_accuracy(predictions, targets)
print("Accuracy: %0.2f" % magnitude_accuracy)

Accuracy: 0.69


#### Make a barplot of the two results.

In [15]:
# What to do here?

## (Q1.3 Optional) A better threshold (1pt)
Above we have defined a threshold to account for an inherent bias in the dataset: there are more positive than negative words per review.
However, that threshold does not take into account *document length*. Explain why this is a problem and implement an alternative way to compute the threshold.

When we have a review of 7 words, the change of them all being positive is very small. But it can still by a positive review. 
hence we need to normalize with respect to the review length.

A percentage could be a good way to normalize we then get:

....

An alternative approach is to group into x amount of thresholds each for 


In [16]:
## calculate the average percentage of positive words (compared to pos and neg words)
avg = 0
total = 0

for sentence in sentences:
    pos_count = 0
    neg_count  = 0
    for word in sentence:
        s = np.sign(slex[word]) 

        if  s < 0:
            neg_count += 1
        if s > 0:
            pos_count += 1
    total += ((pos_count - neg_count) / len(sentence))


avg = total/len(sentences) 
print(avg)



0.009082534363548262


In [17]:
class BinarySentimentPercClassifier:
    
    def __init__(self, slex, threshold):
        self.slex = slex
        self.threshold = threshold

    def predict(self, sentences):
        return [int( self.binary_sentiment(sentence) > ( self.threshold * len(sentence) )) for sentence in sentences ]
    
    def binary_sentiment(self, sentence):
        '''
        sentence: sentence to calculate the sentiment from
        returns the binary sentiment score of a sentence given the sentiment lexicon
        '''

        return np.sum([ np.sign(self.slex[word])   for word in sentence])
                    
perc_model = BinarySentimentPercClassifier(slex, avg)

predictions = perc_model.predict(sentences)
perc_results = predictions
perc_accuracy = calculate_accuracy(predictions, targets)
print("Accuracy: %0.2f" % perc_accuracy)

Accuracy: 0.66


In [18]:
## calculate the average percentage of positive words (compared to pos and neg words)

threshold_dict = {
    "small": 0,
    "medium": 0,
    "large": 0
}
print("yes")

for sentence in sentences:
    pos_count = 0
    neg_count  = 0
    for word in sentence:
        s = np.sign(slex[word]) 

        if  s < 0:
            neg_count += 1
        if s > 0:
            pos_count += 1
    total += ((pos_count - neg_count) / len(sentence))

yes


# Significance testing

Does using the magnitude improve the results? Oftentimes, answering such questions by simply comparing accuracy
scores is not enough. When dealing with natural language and human ratings, it is safe to assume that there are infinitely many possible
instances that could be used for training and testing, of which the ones
we actually train and test on are a tiny sample. Thus, it is possible
that observed differences in the reported performance are due to mere chance.

There exist statistical methods which can be used to check for
consistency (*statistical significance*) in the results, and one of the
simplest such tests is the **sign test**. 

The sign test is based on the binomial distribution. Count all cases when System 1 is better than System 2, when System 2 is better than System 1, and when they are the same. Call these numbers $Plus$, $Minus$ and $Null$ respectively. 

The sign test returns the probability that the null hypothesis is true. 

This probability is called the $p$-value and it can be calculated for the two-sided sign test using the following formula (we multiply by two because this is a two-sided sign test and tests for the significance of differences in either direction):

$$2 \, \sum\limits_{i=0}^{k} \binom{N}{i} \, q^i \, (1-q)^{N-i}$$

where $$N = 2 \Big\lceil \frac{Null}{2}\Big\rceil + Plus + Minus$$ is the total
number of cases, and
$$k = \Big\lceil \frac{Null}{2}\Big\rceil + \min\{Plus,Minus\}$$ is the number of
cases with the less common sign. 

Here, we
treat ties by adding half a point to either side, rounding up to the
nearest integer if necessary. 

In this experiment, $q = 0.5$, so the formula simplifies to:
$$2\times 0.5^N\, \sum\limits_{i=0}^{k} \binom{N}{i}$$


We use the `comb` function from `scipy` and the `decimal` package for the stable adding of numbers in the final summation.


In [19]:
from decimal import Decimal
from scipy.special import comb


def sign_test(results_1, results_2):
  """test for significance"""
  ties, plus, minus = 0, 0, 0

  for i in range(0, len(results_1)):
    if results_1[i]==results_2[i]:
      ties += 1
    elif results_1[i]==0: 
      plus += 1
    elif results_2[i]==0: 
      minus += 1

  n = (2 * math.ceil(ties/2.0)) + plus + minus
  k = math.ceil(ties/2.0) + min(plus,minus)

  summation = Decimal(0.0)
  for i in range(0,int(k)+1):
      summation += (Decimal(comb(n,i,exact=True)))

  # use two-tailed version of test
  summation *= 2
  summation *= (Decimal(0.5)**Decimal(n))
  
  print("the difference is", 
        "not significant" if summation >= 0.05 else "significant")
  
  return summation

p_value = sign_test(token_results, magnitude_results)
print("p_value =", p_value)

the difference is not significant
p_value = 0.5024515744752503461039016873


### Using the Sign test

**From now on, report all differences between systems, as well as between system configurations, using the
sign test.**
    
You should report statistical test
results in an appropriate form. If there are several different methods
(i.e., systems) to compare, the Sign test can only be applied to pairs of them
at a time. When reporting these pair-wise differences, you should
summarise trends to avoid redundancy.


# Naive Bayes (10pt)


Your second task is to program a simple Machine Learning approach that operates
on a simple Bag-of-Words (BoW) representation of the text data, as
described by Pang et al. (2002). In this approach, the only features we
will consider are the words in the text themselves, without bringing in
external sources of information. The BoW model is a popular way of
representing texts as vectors, making it
easy to apply classical Machine Learning algorithms on NLP tasks.
However, the BoW representation is also very crude, since it discards
all information related to word order and grammatical structure in the
original text—as the name suggests.

## Writing your own classifier (4pts)

Write your own code to implement the Naive Bayes (NB) classifier. As
a reminder, the Naive Bayes classifier works according to the following
equation:
$$\hat{c} = \operatorname*{arg\,max}_{c \in C} P(c|\bar{f}) = \operatorname*{arg\,max}_{c \in C} P(c)\prod^n_{i=1} P(f_i|c)$$
where $C = \{ \text{POS}, \text{NEG} \}$ is the set of possible classes,
$\hat{c} \in C$ is the most probable class, and $\bar{f}$ is the feature
vector. Remember that we use the log of these probabilities when making
a prediction:
$$\hat{c} = \operatorname*{arg\,max}_{c \in C} \Big\{\log P(c) + \sum^n_{i=1} \log P(f_i|c)\Big\}$$

You can find more details about Naive Bayes in [Jurafsky &
Martin](https://web.stanford.edu/~jurafsky/slp3/). You can also look at
this helpful
[pseudo-code](https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html).

*Note: this section and the next aim to put you in a position to replicate
    Pang et al.'s Naive Bayes results. However, your numerical results
    will differ from theirs, as they used different data.*

**You must write the Naive Bayes training and prediction code from
scratch.** You will not be given credit for using off-the-shelf Machine
Learning libraries.

The data contains the text of the reviews, where each document consists
of the sentences in the review, the sentiment of the review and an index
(cv) that you will later use for cross-validation. The
text has already been tokenised and POS-tagged for you. Your algorithm
should read in the text, **lowercase it**, store the words and their
frequencies in an appropriate data structure that allows for easy
computation of the probabilities used in the Naive Bayes algorithm, and
then make predictions for new instances.


#### (Q3.1) Unseen words (1pt)
The presence of words in the test dataset that
have not been seen during training can cause probabilities in the Naive Bayes classifier to equal $0$.
These can be words which are unseen in both positive and negative training reviews (case 1), but also words which are seen in reviews _of only one sentiment class_ in the training dataset (case 2). In both cases, **you should skip these words for both classes**. 

In case 2, you could also set $P(c|\bar{f}) = 0$ for the class $c$ within which $f_i \in \bar{f}$ was not seen. One way to implement this in log space is to set $P(c|\bar{f}) = -\infty$. What would be the problem instead with skipping words only for one class in case 2? 

#### answer 
We have two classes: $c_1$ and $c_2$

Let $c_1$ be the class for which $f_i$ was seen and $c_2$ be the class for which $f_i$ was not seen. Assume that all other features were observed by both classes.

Then $P(c_1| \bar{f}) > P(c_2| \bar{f})$, regardless of the other features! This is undeserable, because maybe when we skipped that one feature to get $\bar{f}'$ we would have had: $P(c_1| \bar{f}') < P(c_2| \bar{f}')$

Hence just because one word only showed up in one class, doesn't make that class the most probably candidate.


#### (Q3.2) Train your classifier on (positive and negative) reviews with cv-value 000-899, and test it on the remaining (positive and negative) reviews cv900–cv999.  Report results using classification accuracy as your evaluation metric. Your  features are the word vocabulary. The value of a feature is the count of that feature (word) in the document. (2pts)


In [20]:
def create_dataset_split(reviews, cv_train, cv_test):
    train_targets = []
    test_targets = []
    train_sentences = []
    test_sentences = []
    
    for review in reviews:
        if review['cv'] in cv_train:
            train_targets.append(int( review['sentiment'] == 'POS'))
            whole_review = create_whole_review( review['content'])
            train_sentences.append(whole_review)
        if review['cv'] in cv_test:
            
            test_targets.append(int( review['sentiment'] == 'POS'))
            whole_review = create_whole_review( review['content'])
            test_sentences.append(whole_review)

    return train_sentences, train_targets, test_sentences, test_targets




In [21]:
from tqdm import tqdm
def create_voc(sentences):
    '''
        Creates a vocubulair of the sentences
        Uses lowercase
        returns list of words.
    '''
    voc = list(set([word for sentence in sentences for word in sentence]))

    return voc
    

class NaiveBayesClassifier:
    
    def __init__(self, k=0):
        self.log_likelihood = defaultdict(float)
        self.log_prior = defaultdict(float)
        self.voc = None
        self.counts = defaultdict(float)
        self.k = k
    
    def fit(self, sentences, targets):
        '''
            Fits the naive bayes classifier, given the sentences and the targets
        '''
        n_doc = len(sentences)
        n_pos = np.sum(targets)
        n_neg = n_doc - n_pos
        
        
        self.log_prior = {
            0: np.log(n_neg/n_doc),
            1: np.log(n_pos/n_doc)
        } 
        


        self.voc = create_voc(sentences)
        
        pos_idx = np.array(targets) == 1
        doc = {
            0: np.array(sentences, dtype=object)[~pos_idx],
            1:  np.array(sentences, dtype=object)[pos_idx]
        }
        

        for c, sentences in doc.items():
            for sentence in sentences:
                for w in sentence:
                    self.counts[(w,c)] += 1
        
        
        total_counts = {
            0: np.sum([self.counts[(w,0)] + self.k  for w in self.voc]),
            1: np.sum([self.counts[(w,1)] + self.k  for w in self.voc])
        }

        for c in doc.keys():
            if total_counts[c] != 0:
                for w in self.voc:
                    if self.counts[(w,c)] + self.k > 0:
                        self.log_likelihood[(w, c)] = np.log( (self.counts[(w,c)] + self.k )/ total_counts[c]) 

    
    def predict(self, sentences):
        '''
            Predict the label of the given sentences
        '''
        result = []
        for sentence in tqdm(sentences):
            result.append(self.predict_1(sentence))
        return result
    
    def predict_1(self, sentence):
        '''
            Predicts the label of one sentence
        '''
        
        sum_c = [self.log_prior[0], self.log_prior[1]]
        for c in [0,1]:
            for word in sentence:
                    ### Note that we implicitally use that words that did not appear do not count, by using a default dict
                    # We need to make sure the words are lowercase
                    sum_c[c] += self.log_likelihood[(word, c)]
                    
        return np.argmax(sum_c)

    def reset_params(self):
        '''
        Resets the parameters, such that we can do another round of trainig.
        '''
        self.log_likelihood = defaultdict(float)
        self.log_prior = defaultdict(float)
        self.voc = None
        self.counts = defaultdict(float)
        
    
train_sentences, train_targets, test_sentences, test_targets = create_dataset_split(reviews, [i for i in range(0,900)], [i for i in range(900,1000)])
naive_bayes_model = NaiveBayesClassifier()
naive_bayes_model.fit(train_sentences, train_targets)

In [22]:
predictions = naive_bayes_model.predict(test_sentences)

naive_bayes_results = calculate_results(predictions, test_targets)
naive_bayes_accuracy = calculate_accuracy(predictions, test_targets)
print("Accuracy: %0.2f" % naive_bayes_accuracy)

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 2061.33it/s]

Accuracy: 0.49


#### (Q3.3) Would you consider accuracy to also be a good way to evaluate your classifier in a situation where 90% of your data instances are of positive movie reviews? (1pt)

Simulate this scenario by keeping the positive reviews
data unchanged, but only using negative reviews cv000–cv089 for
training, and cv900–cv909 for testing. Calculate the classification
accuracy, and explain what changed.

The classification accuracy becomes 0.0, because the training set only contains negative examples and no positive examples, hence the log prior for the positive class will become $-\infty$

In [23]:
train_sentences, train_targets, test_sentences, test_targets = create_dataset_split(reviews, [i for i in range(0,90)], [i for i in range(900,910)])
index_train = 90
index_test = 9

### we need to make sure tha we only get the positive/negative examples from those indexes
train_targets = train_targets[:index_train]
train_sentences = train_sentences[:index_train]

test_targets = test_targets[-index_test:]
test_sentences = test_sentences[-index_test:]

all_neg_naive_bayes_model = NaiveBayesClassifier()
all_neg_naive_bayes_model.fit(train_sentences, train_targets)
predictions = all_neg_naive_bayes_model.predict(test_sentences)


naive_bayes_all_neg_results = calculate_results(predictions, test_targets)

naive_bayes_all_neg_accuracy = calculate_accuracy(naive_bayes_all_neg_results, test_targets)
print("Accuracy: %0.2f" % naive_bayes_all_neg_accuracy)

C:\Users\gerso\anaconda3\envs\nlp1\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in log
100%|██████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 1806.68it/s]

Accuracy: 0.00


## Smoothing (1.5pts)

As mentioned above, the presence of words in the test dataset that
have not been seen during training can cause probabilities in the Naive
Bayes classifier to be $0$, thus making that particular test instance
undecidable. The standard way to mitigate this effect (as well as to
give more clout to rare words) is to use smoothing, in which the
probability fraction
$$\frac{\text{count}(w_i, c)}{\sum\limits_{w\in V} \text{count}(w, c)}$$ for a word
$w_i$ becomes
$$\frac{\text{count}(w_i, c) + \text{smoothing}(w_i)}{\sum\limits_{w\in V} \text{count}(w, c) + \sum\limits_{w \in V} \text{smoothing}(w)}$$





#### (Q3.4) Implement Laplace feature smoothing (1pt)
Implement Laplace smoothing, i.e., smoothing with a constant value ($smoothing(w) = \kappa, \forall w \in V$), in your Naive
Bayes classifier’s code, and report the impact on performance. 
Use $\kappa = 1$.

In [24]:

train_sentences, train_targets, test_sentences, test_targets = create_dataset_split(reviews, [i for i in range(0,900)], [i for i in range(900,1000)])
smooth_naive_bayes_model = NaiveBayesClassifier(1)
smooth_naive_bayes_model.fit(train_sentences, train_targets)

In [25]:
predictions = smooth_naive_bayes_model.predict(test_sentences)

smoothed_naive_bayes_results = calculate_results(predictions, test_targets)
smoothed_naive_bayes_accuracy = calculate_accuracy(predictions, test_targets)
print("Accuracy: %0.2f" % smoothed_naive_bayes_accuracy)

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 1869.28it/s]

Accuracy: 0.82


#### (Q3.5) Is the difference between non smoothed (Q3.2) and smoothed (Q3.4) statistically significant? (0.5pt)

In [26]:

p_value = sign_test(smoothed_naive_bayes_results, naive_bayes_results)
print("p_value =", p_value)


the difference is significant
p_value = 0.000003547178174130642586494974890


The difference is statistically significant, which is to be expected as the smoothed version has a much higher accuracy than the not smoothed version. 

## Cross-Validation (1.5pts)

A serious danger in using Machine Learning on small datasets, with many
iterations of slightly different versions of the algorithms, is ending up with Type III errors, also called the “testing hypotheses
suggested by the data” errors. This type of error occurs when we make
repeated improvements to our classifiers by playing with features and
their processing, but we don’t get a fresh, never-before seen test
dataset every time. Thus, we risk developing a classifier that gets better
and better on our data, but only gets worse at generalizing to new, unseen data. In other words, we risk developping a classifier that overfits.

A simple method to guard against Type III errors is to use
Cross-Validation. In **N-fold Cross-Validation**, we divide the data into N
distinct chunks, or folds. Then, we repeat the experiment N times: each
time holding out one of the folds for testing, training our classifier
on the remaining N - 1 data folds, and reporting performance on the
held-out fold. We can use different strategies for dividing the data:

-   Consecutive splitting:
  - cv000–cv099 = Split 1
  - cv100–cv199 = Split 2
  - etc.
  
-   Round-robin splitting (mod 10):
  - cv000, cv010, cv020, … = Split 1
  - cv001, cv011, cv021, … = Split 2
  - etc.

-   Random sampling/splitting
  - Not used here (but you may choose to split this way in a non-educational situation)



#### (Q3.6) Write the code to implement 10-fold cross-validation using round-robin splitting for your Naive Bayes classifier from Q3.2 and compute the 10 accuracies. Report the final performance, which is the average of the performances per fold. If all splits perform equally well, this is a good sign. (1pt)

In [27]:
def create_folds(data, n_folds):
    review_folds = [[] for i in range(n_folds)]
    label_folds = [[] for i in range(n_folds)]

    for review in data:
        i = review['cv'] % n_folds

        whole_review = create_whole_review( review['content'])
        review_folds[i].append(whole_review)
        label_folds[i].append(int( review['sentiment'] == 'POS'))
    return review_folds, label_folds

        
review_folds, label_folds = create_folds(reviews, 10)


In [28]:
def train_on_fold(model, x_folds, y_folds, held_out_index):
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    for i in range(len(x_folds)):
        if i != held_out_index:
            train_x += x_folds[i]
            train_y += y_folds[i]
        else:
            test_x += x_folds[i]
            test_y += y_folds[i]
    
    model.fit(train_x, train_y)
    predictions = model.predict(test_x)
    results = calculate_results(predictions, test_y)
    accuracy  = calculate_accuracy(predictions, test_y)
    return accuracy, predictions, results

def train_on_folds(create_model, x_folds, y_folds):
    models = [create_model() for i in range(len(x_folds))]
    accuracies = []
    predictions = []
    results = []
    for i, fold in enumerate(x_folds):
        model = models[i]
        accuracy, prediction, result = train_on_fold(model, x_folds, y_folds, i)
        
        accuracies.append(accuracy)
        predictions.append(predictions)
        results.append(result)
    return models, accuracies, predictions, results
create_model = lambda: NaiveBayesClassifier(1)
models, accuracies, predictions, results = train_on_folds(create_model, review_folds, label_folds)

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 2106.33it/s]


In [29]:
print(models)
print(accuracies)

[<__main__.NaiveBayesClassifier object at 0x0000027926680448>, <__main__.NaiveBayesClassifier object at 0x0000027926680348>, <__main__.NaiveBayesClassifier object at 0x0000027926680248>, <__main__.NaiveBayesClassifier object at 0x00000279266F4D08>, <__main__.NaiveBayesClassifier object at 0x00000279266F4B88>, <__main__.NaiveBayesClassifier object at 0x00000279266F5F08>, <__main__.NaiveBayesClassifier object at 0x000002792DCA5788>, <__main__.NaiveBayesClassifier object at 0x000002792DCA57C8>, <__main__.NaiveBayesClassifier object at 0x000002792DCA5808>, <__main__.NaiveBayesClassifier object at 0x000002792147DF08>]
[0.79, 0.835, 0.805, 0.825, 0.78, 0.845, 0.83, 0.775, 0.83, 0.84]


#### (Q3.7) Write code to calculate and report the variance of the 10 accuracy scores, in addition to the final performance. You must not use a library, such as numpy. (0.5pt)

**Please report all future results using 10-fold cross-validation now
(unless told to use the held-out test set).**

In [30]:
def compute_mean(values):

    total = 0
    for val in values:
        total += val
    return total/len(values)

def compute_std(values):
    ## Calculate the std:
    mean = compute_mean(values)
    tmp_sum = 0
    for val in values:
        tmp_sum += (val- mean)**2
    return math.sqrt(1/len(values) * tmp_sum )
print(accuracies)
print(compute_mean(accuracies))
print(compute_std(accuracies))


[0.79, 0.835, 0.805, 0.825, 0.78, 0.845, 0.83, 0.775, 0.83, 0.84]
0.8155000000000001
0.024540782383616025


## Features, overfitting, and the curse of dimensionality

In the Bag-of-Words model, ideally we would like each distinct word in
the text to be mapped to its own dimension in the output vector
representation. However, real world text is messy, and we need to decide
on what we consider to be a word. For example, is “`word`" different
from “`Word`", from “`word`”, or from “`words`"? Too strict a
definition, and the number of features explodes, while our algorithm
fails to learn anything generalisable. Too lax, and we risk destroying
our learning signal. In the following section, you will learn about
confronting the feature sparsity and the overfitting problems as they
occur in NLP classification tasks.

### Stemming (1.5pts)

To make your algorithm more robust, use stemming and
hash different inflections of a word to the same feature in the BoW
vector space. How does the performance of your classifier change when
you use stemming on your training and test datasets? Please use the [Porter stemming
    algorithm](http://www.nltk.org/howto/stem.html) from NLTK.

You should also perform cross-validation. Concatenate the predictions from all folds to compute the significance.

In [31]:
from nltk.stem import *

def stem_sentences(sentences):
    
    port_stemmer = PorterStemmer()
    snowball_stemmer = SnowballStemmer("english")
    result = []
    
    for sentence in sentences:
        result.append([port_stemmer.stem(snowball_stemmer.stem(word)) for word in sentence])
        
    return result


stemmed_x_folds = [stem_sentences(x) for x in review_folds]



In [32]:
stemmed_models, stemmed_accuracies, stemmed_predictions, stemmed_results  = train_on_folds(create_model, stemmed_x_folds, label_folds)

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 2380.96it/s]


In [33]:
print(accuracies)
print(compute_mean(accuracies))
print(compute_std(accuracies))

[0.79, 0.835, 0.805, 0.825, 0.78, 0.845, 0.83, 0.775, 0.83, 0.84]
0.8155000000000001
0.024540782383616025


#### (Q3.9): Is the difference between NB with smoothing and NB with smoothing+stemming significant? (1pt)


In [34]:
# How to test this? Which dataset do we need to use?


#### (Q3.10) What happens to the number of features (i.e., the size of the vocabulary) when using stemming as opposed to (Q3.4)? (0.5pt)
Give actual numbers. You can use the held-out training set to determine these.

In [35]:
print(len(models[0].voc))
print(len(stemmed_models[0].voc))

45329
31591


### N-grams (1.5pts)

A simple way of retaining some of the word
order information when using bag-of-words representations is to add **n-gram** features. 






#### (Q3.11) Retrain your classifier from (Q3.4) using **unigrams+bigrams** and **unigrams+bigrams+trigrams** as features. (1pt)
Report accuracy and statistical significances (in comparison to the experiment at (Q3.6) for all 10 folds, and between the new systems). You are allowed to use NLTK to build n-grams from sentences.

In [36]:
from nltk import ngrams


def to_n_grams(sentences, ns=[1,23]):
    
    
    result = []
    
    for sentence in sentences:
        temp = []
        for n in ns:
            temp += ngrams(sentence, n)
            
        result.append(temp)
        
    return result


un_bi_gram_x = [to_n_grams(x, ns=[1,2]) for x in review_folds]


In [37]:
un_bi_models, un_bi_accuracies, un_bi_predictions, un_bi_results  = train_on_folds(create_model, un_bi_gram_x, label_folds)

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 595.12it/s]


In [38]:
print(len(un_bi_models[0].voc))

472632


In [39]:
print(un_bi_accuracies)
print(compute_mean(un_bi_accuracies))

[0.795, 0.85, 0.84, 0.875, 0.81, 0.86, 0.83, 0.83, 0.845, 0.835]
0.8370000000000001


In [40]:
un_bi_tri_gram_x = [to_n_grams(x, ns=[1,2, 3]) for x in review_folds]
un_bi_tri_models, un_bi_tri_accuracies, un_bi_tri_predictions, un_bi_tri_results  = train_on_folds(create_model, un_bi_tri_gram_x, label_folds)

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 324.15it/s]


In [41]:
print(un_bi_tri_accuracies)
print(compute_mean(un_bi_tri_accuracies))

[0.79, 0.85, 0.835, 0.86, 0.82, 0.85, 0.85, 0.84, 0.85, 0.805]
0.835



#### Q3.12: How many features does the BoW model have to take into account now? (0.5pt)
How would you expect the number of features to increase theoretically (e.g., linear, square, cubed, exponential)? How does this number compare, in practice, to the number of features at (Q3.10)?

Use the held-out training set once again for this.


Theoretically I would expect that the number of features for O(m^n) where m is the number of unigrams and n is the highest ngram that we use. Because when we use n-grams with m words there are m^n possible different n_grams
A lot of the n_grams do not appear in practice hence we would expect to have, in practice, a lot less feautures.
As you can see from the prints below, a tiny fraction of the actual number of features is shown in the model that uses un, bi and tri-grams.


In [42]:
print("unigram features:")
print(len(models[0].voc))
print("expected tri gram features: ")
print(len(models[0].voc) ** 3)
print("actual number of tri gram features: ")
print(len(un_bi_tri_models[0].voc))
print("observed percentage: ")
print( len(un_bi_tri_models[0].voc) / len(models[0].voc) ** 3 )


unigram features:
45329
expected tri gram features: 
93138323146289
actual number of tri gram features: 
1423850
observed percentage: 
1.5287477290777613e-08


# Support Vector Machines (4pts)

Though simple to understand, implement, and debug, one
major problem with the Naive Bayes classifier is that its performance
deteriorates (becomes skewed) when it is being used with features which
are not independent (i.e., are correlated). Another popular classifier
that doesn’t scale as well to big data, and is not as simple to debug as
Naive Bayes, but that doesn’t assume feature independence is the Support
Vector Machine (SVM) classifier.

You can find more details about SVMs in Chapter 7 of Bishop: Pattern Recognition and Machine Learning.
Other sources for learning SVM:
* http://web.mit.edu/zoya/www/SVM.pdf
* http://www.cs.columbia.edu/~kathy/cs4701/documents/jason_svm_tutorial.pdf
* https://pythonprogramming.net/support-vector-machine-intro-machine-learning-tutorial/







Use the scikit-learn implementation of 
[SVM](http://scikit-learn.org/stable/modules/svm.html) with the default parameters. (You are not expected to perform any hyperparameter tuning, but feel free to do it if you think it gives you good insights for the discussion in question 5.)



#### (Q4.1): Train SVM and compare to Naive Bayes (2pts)

Train an SVM classifier (sklearn.svm.LinearSVC) using your features. Compare the
classification performance of the SVM classifier to that of the Naive
Bayes classifier with smoothing from (Q3.6) and report the numbers.
Perform cross-validation and concatenate the predictions from all folds to compute the significance.  Are the results significantly better?



In [48]:

### First we need to create one_hot vectors for the features:


def voc_to_one_hot(voc):
    one_hot_mapping = {}
    for i in tqdm(range(len(voc))):
        word = voc[i]
        one_hot_mapping[word] = np.zeros((len(voc)))
        one_hot_mapping[word][i] += 1
    return one_hot_mapping

def create_voc_of_reviews(reviews):
    sentences = []
    for review in reviews:
        whole_review = create_whole_review( review['content'])
        sentences.append(whole_review)
    voc = create_voc(sentences)
    return voc


voc = create_voc_of_reviews(reviews)
one_hot_mapping = voc_to_one_hot(voc)





100%|███████████████████████████████████████████████████████████████████████████| 47743/47743 [01:24<00:00, 562.12it/s]


In [52]:
print(one_hot_mapping["a"])
print(np.max(one_hot_mapping["a"]))

[0. 0. 0. ... 0. 0. 0.]
1.0


In [105]:
### Next we do a train_test_split:
train_sentences, train_targets, test_sentences, test_targets = create_dataset_split(reviews, [i for i in range(0,10)], [i for i in range(900,1000)])

In [106]:
### Then we map each sentence
to_one_hot = lambda sentence: [one_hot_mapping[word] for word in sentence]
train_sentences = [to_one_hot(sentence) for sentence in train_sentences]
#test_sentences = [to_one_hot(sentence) for sentence in test_sentences]

In [107]:
one_hot_train_sentences = []
for i in tqdm(range(len(train_sentences))):
    sentence = train_sentences[i]
    one_hot_train_sentences.append(np.sum(sentence, axis=0))
print(one_hot_train_sentences[0])

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  8.92it/s]

[0. 0. 0. ... 0. 0. 0.]


In [109]:
from sklearn import svm
clf = svm.SVC()
clf.fit(one_hot_train_sentences, train_targets)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0])

### POS disambiguation (2pts)

Now add in part-of-speech features. You will find the
movie review dataset has already been POS-tagged for you ([here](https://catalog.ldc.upenn.edu/docs/LDC99T42/tagguid1.pdf) you find the tagset). Try to
replicate the results obtained by Pang et al. (2002).



####(Q4.2) Replace your features with word+POS features, and report performance with the SVM. Does this help? Perform cross-validation and concatenate the predictions from all folds to compute the significance. Are the results significant? *Why?*  (1pt)


In [ ]:
# YOUR CODE HERE

*Write your answer here.*

#### (Q4.3) Discard all closed-class words from your data (keep only nouns, verbs, adjectives, and adverbs), and report performance. Does this help? Perform cross-validation and concatenate the predictions from all folds to compute the significance. Are the results significantly better than when we don't discard the closed-class words? *Why?* (1pt)

In [ ]:
# YOUR CODE HERE

*Write your answer here.*

# (Q5) Discussion (max. 500 words). (5pts)

> Based on your experiments, what are the effective features and techniques in sentiment analysis? What information do different features encode?
Why is this important? What are the limitations of these features and techniques?
 


*Write your answer here in up to 500 words (-0.25pt for >50 extra words, -0.5 points for >100 extra words, ...)*.


# Submission 


In [ ]:
# Write your names and student numbers here:
# Student 1 #12345
# Student 2 #12345

**That's it!**

- Check if you answered all questions fully and correctly. 
- Download your completed notebook using `File -> Download .ipynb` 
- Check if your answers are all included in the file you submit.
- Submit your .ipynb file via *Canvas*. One submission per group. 